# Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Se tiene acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). 

# Descripción de datos

- `/datasets/users_behavior.csv '` contiene los siguientes datos:
  
   -`сalls:` número de llamadas
     
   -`minutes` — duración total de la llamada en minutos
  
   -`messages` — número de mensajes de texto
  
   -`mb_used` — Tráfico de Internet utilizado en MB
  
   -`is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0)

# Librerías

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

## Cargar datos

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Preparar los datos

In [3]:
nulos = data.isnull().sum()
print(nulos)
print()
print('El dataset tiene', data.duplicated().sum(),'duplicados')
print()
data.info()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

El dataset tiene 0 duplicados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data['messages'] = data['messages'].astype('int64')


Los datos del conjunto de datos se encuentran completos, correctamente estructurados y no requirieron modificaciones. No obstante, la columna 'messages' fue convertida al tipo de dato 'int64' como buena práctica, considerando que no existe la posibilidad de que se generen valores decimales.

In [5]:
count = data['is_ultra'].value_counts()
count

0    2229
1     985
Name: is_ultra, dtype: int64

El conjunto de datos muestra un desbalance moderado, con una proporción de 30,64% para la clase 1 y 69,35% para la clase 0, lo cual podría afectar el rendimiento de algunos clasificadores.

# Modelos de clasificación

Se separa el dataset en una relación 3:1, es decir, el 75% de los datos se utilizan para el entrenamiento del modelo (df_train) y el 25% restante para validación o prueba (df_valid).Como variable objetivo (target) se selecciona la columna 'is_ultra', y como características (features) se utilizan las columnas restantes: 'call', 'minutes', 'messages' y 'mb_used'.

In [6]:
# separción  de los datos de entrenamiento y validación
df_train, df_valid = train_test_split(data, test_size=0.25, random_state=42)

# Declaración de las variables de las caracteristicas  y los objeticos
#Entrenamiento
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
#Validación
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [7]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(2410, 4)
(2410,)
(804, 4)
(804,)


Para la prueba de cordura, se busca verificar si el modelo realmente está aprendiendo algo significativo. Dado que el conjunto de datos presenta un desbalance aproximado del 70%-30%, un modelo que simplemente prediga siempre la clase mayoritaria (clase 0) alcanzaría una accuracy del 70% sin aportar valor real.
Por esta razón, esta prueba es fundamental para determinar si el modelo está aprendiendo patrones útiles o si solo está aprovechando el desbalance de clases.
Para ello, se calcula un dummy accuracy, que corresponde a la proporción de la clase más frecuente en los datos, y se compara con la accuracy del modelo. Si el modelo no supera esta métrica, se considera que no ha aprendido nada mejor que una predicción trivial.

In [8]:
#test para prueba de cordura
dummy_accuracy= target_valid.value_counts(normalize=True).max()

## Regresión Logística

In [9]:
log_reg = LogisticRegression(random_state=42,solver='liblinear')
log_reg.fit(features_train,target_train)


LogisticRegression(random_state=42, solver='liblinear')

In [10]:
lr_prediction = log_reg.predict(features_valid)
prefiction_lr_1= log_reg.predict_proba(features_valid)[:,1]

print(f'Accuracy: {accuracy_score(target_valid, lr_prediction):.5f}')
print(f'precisión:{precision_score(target_valid, lr_prediction):.5f}')
print(f'Recall:{recall_score(target_valid, lr_prediction):.5f}')
print(f'F1_score:{f1_score(target_valid, lr_prediction):.5f}')
print(f'ROC AUC clase 1: {roc_auc_score(target_valid, prefiction_lr_1 ):.5f}')

Accuracy: 0.72388
precisión:0.86957
Recall:0.08368
F1_score:0.15267
ROC AUC clase 1: 0.59199


In [11]:
# Prueba de cordura
lr_acurracy = accuracy_score(target_valid, lr_prediction)
if lr_acurracy <= dummy_accuracy:
    print("¡Tu modelo no supera a un clasificador tonto!")
else:
    print("El modelo pasa la prueba de cordura.")

El modelo pasa la prueba de cordura.


Tras varias modificaciones a los hiperparámetros, no se logró que el modelo de regresión logística alcanzara el umbral de exactitud 0.75. Sin embargo, es importante destacar que el modelo presenta una precisión alta (0.86), lo que indica que, cuando predice un positivo, suele acertar.No obstante, el recall es muy bajo, lo que revela que el modelo casi no logra detectar verdaderos positivos, es decir, deja pasar la mayoría de los casos relevantes. Esta combinación de alta precisión y bajo recall resulta en un F1-score bajo (0.15), lo cual refleja un rendimiento general deficiente para tareas donde es crítico identificar todos los casos positivos. Adicionalmente, el valor del Área bajo la curva ROC (AUC-ROC) es de 0.59, lo que sugiere que el modelo apenas supera el rendimiento esperado por azar (0.5) y no logra discriminar adecuadamente entre clases.

## RandomForestClassifier

In [12]:
best_score = 0
best_est = 0

for est in range(1, 100,5): 
    model_rf = RandomForestClassifier(random_state=42, n_estimators=est) 
    model_rf.fit(features_train, target_train) 
    score = model_rf.score(features_valid,target_valid) 
    if score > best_score:
        best_score = score
        best_est = est

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 41): 0.8283582089552238


In [13]:
rf = RandomForestClassifier(random_state=42, n_estimators=42,criterion='gini')
rf.fit(features_train,target_train)

prediction_rf = rf.predict(features_valid)
prediction_rf_1 = rf.predict_proba(features_valid)[:,1]

print(f'Accuracy: {accuracy_score(target_valid, prediction_rf):.5f}')
print(f'precisión:{precision_score(target_valid, prediction_rf):.5f}')
print(f'Recall:{recall_score(target_valid, prediction_rf):.5f}')
print(f'F1_score:{f1_score(target_valid, prediction_rf):.5f}')
print(f'ROC AUC clase 1: {roc_auc_score(target_valid, prediction_rf_1):.5f}')

Accuracy: 0.82960
precisión:0.78652
Recall:0.58577
F1_score:0.67146
ROC AUC clase 1: 0.80021


In [14]:
# Prueba de cordura
rf_acurracy = accuracy_score(target_valid, prediction_rf)
if lr_acurracy <= dummy_accuracy:
    print("¡Tu modelo no supera a un clasificador tonto!")
else:
    print("El modelo pasa la prueba de cordura.")

El modelo pasa la prueba de cordura.


El modelo evaluado supera el umbral de exactitud establecido en 0.75, alcanzando un accuracy de 0.83, lo cual refleja un buen rendimiento general. No obstante, debido al desbalance presente en el conjunto de datos, la exactitud por sí sola no resulta suficiente para evaluar adecuadamente su desempeño. La precisión de 0.78652 indica que, cuando el modelo predice la clase positiva (1), suele acertar con alta frecuencia. Por otro lado, el recall de 0.58577 sugiere que el modelo es capaz de identificar una proporción moderada de los casos positivos reales, aunque deja pasar una cantidad considerable. En consecuencia, el F1-score de 0.67146, que representa un equilibrio entre precisión y recall, es aceptable.Finalmente, el ROC AUC de 0.80 para la clase 1 demuestra que el modelo posee una buena capacidad de discriminación entre clases, incluso en contextos de desbalance.

## KNeighborsClassifier

In [15]:
knn = KNeighborsClassifier(n_neighbors=5, algorithm='kd_tree')
knn.fit(features_train,target_train)

prediction_knn = knn.predict(features_valid)
prediction_knn_1 = knn.predict_proba(features_valid)[:,1]

print(f'Accuracy: {accuracy_score(target_valid, prediction_knn):.5f}')
print(f'precisión:{precision_score(target_valid, prediction_knn):.5f}')
print(f'Recall:{recall_score(target_valid, prediction_knn):.5f}')
print(f'F1_score:{f1_score(target_valid, prediction_knn):.5f}')
print(f'ROC AUC clase 1: {roc_auc_score(target_valid, prediction_knn_1 ):.5f}')

Accuracy: 0.78980
precisión:0.73333
Recall:0.46025
F1_score:0.56555
ROC AUC clase 1: 0.74533


In [16]:
# Prueba de cordura
knn_acurracy = accuracy_score(target_valid, prediction_knn)
if knn_acurracy <= dummy_accuracy:
    print("El modelo no supera a un clasificador tonto")
else:
    print("El modelo pasa la prueba de cordura.")

El modelo pasa la prueba de cordura.


El modelo K-Nearest Neighbors alcanzó una exactitud de 0.78980, superando ligeramente el umbral establecido de 0.75, lo cual indica un rendimiento general aceptable. Sin embargo, al considerar el desbalance del conjunto de datos, es necesario analizar métricas adicionales. La precisión de 0.73333 revela que, cuando el modelo predice la clase positiva (1), en la mayoría de los casos acierta. No obstante, el recall de 0.46025 indica que el modelo logra identificar menos de la mitad de los positivos reales, lo que sugiere una capacidad limitada para recuperar todos los casos relevantes. Como resultado, el F1-score de 0.56555, que equilibra precisión y recall, refleja un desempeño moderado que podría mejorarse en tareas donde la detección de la clase positiva sea crítica. Por último, el ROC AUC de 0.74533 para la clase 1 muestra una capacidad discriminativa aceptable, aunque inferior a la de otros modelos evaluados, lo que sugiere que el KNN tiene un rendimiento adecuado, pero aún podría optimizarse frente al desbalance existente.

## Árbol de decisiones

In [18]:
dtree = DecisionTreeClassifier(random_state=42, max_depth=42)
dtree.fit(features_train,target_train)

prediction_dtree = dtree.predict(features_valid)
prediction_dtree_1 = dtree.predict_proba(features_valid)[:,1]

print(f'Accuracy: {accuracy_score(target_valid, prediction_dtree):.5f}')
print(f'precisión:{precision_score(target_valid, prediction_dtree):.5f}')
print(f'Recall:{recall_score(target_valid, prediction_dtree):.5f}')
print(f'F1_score:{f1_score(target_valid, prediction_dtree):.5f}')
print(f'ROC AUC clase 1: {roc_auc_score(target_valid, prediction_dtree_1 ):.5f}')


Accuracy: 0.73756
precisión:0.55785
Recall:0.56485
F1_score:0.56133
ROC AUC clase 1: 0.68774


In [ ]:
# Prueba de cordura
dtree_acurracy = accuracy_score(target_valid, prediction_dtree)
if dtree_acurracy <= dummy_accuracy:
    print("El modelo no supera a un clasificador tonto")
else:
    print("El modelo pasa la prueba de cordura.")

El modelo Decision Tree Classifier obtuvo una exactitud de 0.73756, quedando ligeramente por debajo del umbral de referencia de 0.75. Aunque esta métrica sugiere un desempeño general cercano al esperado, en contextos con desbalance de clases (como en este caso), la exactitud no es suficiente para evaluar la calidad del modelo. La precisión de 0.55785 indica que, cuando el modelo predice la clase positiva (1), algo más de la mitad de esas predicciones son correctas. Por otro lado, el recall de 0.56485 muestra que el modelo es capaz de identificar alrededor del 56% de los casos positivos reales, lo cual representa una capacidad de recuperación moderada. El F1-score de 0.56133, que armoniza precisión y recall, refuerza la idea de que el modelo tiene un rendimiento aceptable, aunque sin destacar significativamente en ninguno de los dos aspectos y por ultimo el ROC AUC de 0.68 para la clase 1, que evidencia capacidad media para discriminar entre clases.

## XGBoost

In [19]:
xbg = XGBClassifier(    eval_metric='auc', use_label_encoder=False,random_state=42)
xbg.fit(features_train,target_train)

prediction_xbg = xbg.predict(features_valid)
prediction_xbg_1 = xbg.predict_proba(features_valid)[:,1]

print(f'Accuracy: {accuracy_score(target_valid, prediction_xbg):.5f}')
print(f'precisión:{precision_score(target_valid, prediction_xbg):.5f}')
print(f'Recall:{recall_score(target_valid, prediction_xbg):.5f}')
print(f'F1_score:{f1_score(target_valid, prediction_xbg):.5f}')
print(f'ROC AUC clase 1: {roc_auc_score(target_valid, prediction_xbg_1 ):.5f}')


Accuracy: 0.82338
precisión:0.78363
Recall:0.56067
F1_score:0.65366
ROC AUC clase 1: 0.80732


In [ ]:
# Prueba de cordura
xbg_acurracy = accuracy_score(target_valid, prediction_xbg)
if xbg_acurracy <= dummy_accuracy:
    print("El modelo no supera a un clasificador tonto")
else:
    print("El modelo pasa la prueba de cordura.")

El modelo XGBoost Classifier muestra un rendimiento general sólido, alcanzando una exactitud de 0.82338, superando el umbral de 0.75 establecido como referencia. Este valor sugiere que el modelo clasifica correctamente una proporción alta de los datos. La precisión de 0.78363 indica que, cuando predice la clase positiva (1), acierta en una gran mayoría de los casos, lo cual es valioso en contextos donde los falsos positivos deben minimizarse. El recall de 0.56067, aunque moderado, refleja que el modelo logra identificar un número razonable de los verdaderos positivos, aunque aún hay margen de mejora. El F1-score de 0.65366, al combinar precisión y recall, evidencia un equilibrio decente entre ambas métricas. Finalmente, el ROC AUC de 0.80732 para la clase 1 destaca como uno de los puntos más fuertes del modelo, demostrando una buena capacidad para discriminar entre clases en un conjunto de datos con desbalance.

# Conclusión general

Tras el desarrollo, ajuste y evaluación de los diferentes modelos de clasificación, se identificaron como más adecuados el **RandomForestClassifier** y el **XGBoostClassifier**, los cuales demostraron una mayor capacidad para desempeñar la tarea de forma eficiente. Aunque todos los modelos fueron capaces de manejar razonablemente bien el desbalance moderado del conjunto de datos logrando un ROC AUC mayor a 0.80 para la clase 1, **solo estos dos modelos superaron el umbral de exactitud de 0.75** y presentaron una mejor relación entre precisión, recall y F1-score, lo que indica un balance más efectivo en la predicción de ambas clases.

Se intentó mitigar el desbalance aplicando técnicas como class_weight='balanced' en modelos lineales y scale_pos_weight=700/300 en modelos basados en árboles, pero no se obtuvieron mejoras significativas en el rendimiento. También se consideró la aplicación de SMOTE (Synthetic Minority Over-sampling Technique) como estrategia de sobremuestreo para mejorar la detección de la clase minoritaria, pero las restricciones del entorno virtual de la plataforma impidieron su implementación. A pesar de estas limitaciones, el desempeño alcanzado por los modelos seleccionados es satisfactorio para tareas de clasificación con conjuntos de datos moderadamente desbalanceados.